Biostat/Biomath M257 Homework 4

Due May 17 @ 11:59PM

Tomoki Okuno and 805851067

System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.10.0
Commit 3120989f39b (2023-12-25 18:01 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
  Threads: 2 on 4 virtual cores


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/07_UCLA/Class/257/02_Homework/hw4`


Status `~/Documents/07_UCLA/Class/257/02_Homework/hw4/Project.toml`
  [6e4b80f9] BenchmarkTools v1.5.0
  [944b1d66] CodecZlib v0.7.4
  [8bb1440f] DelimitedFiles v1.9.1
  [0b1a1467] KrylovKit v0.7.1
  [bdcacae8] LoopVectorization v0.12.170
  [b51810bb] MatrixDepot v1.0.11
  [295af30f] Revise v3.5.14
⌃ [2913bbd2] StatsBase v0.33.21
  [b8865327] UnicodePlots v3.6.4
  [37e2e46d] LinearAlgebra
  [2f01184e] SparseArrays v1.10.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [3]:
# remove package Plots
using LinearAlgebra, Random
using BenchmarkTools, SparseArrays
using UnicodePlots, StatsBase

We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

## Q1 (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{align*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{align*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

**Solution**

Let $\mathbf{P} = (p_{ij})$. Under the above rule, we have
$$
\begin{align*}
p_{ij} =
\left\{
    \begin{array}{l}
        \dfrac{pa_{ij}}{r_i} + \dfrac{1-p}{n}　& r_i > 0\\
        \dfrac{1}{n} & r_i = 0
    \end{array}
\right., \quad i, j = 1, \ldots, n.
\end{align*}
$$
Note $\sum_{j=1}^n p_{ij} = 1$. Then $i$-th row vector of $\mathbf{P}$ can be written as
$$
\begin{align*}
\mathbf{p}_i^T = 
    \left\{
        \begin{array}{l}
            \dfrac{p}{r_i}\mathbf{a}_i^T + \dfrac{1-p}{n}\mathbf{1}_n^T　& r_i > 0\\
            \dfrac{1}{n}\mathbf{1}_n^T & r_i = 0
        \end{array}
    \right., \quad i = 1, \ldots n,
\end{align*}
$$
suggesting that $\mathbf{P}$ has a form of
$$
\begin{align*}
    \mathbf{P} = \mathbf{DA} + \mathbf{z}\mathbf 1_n^T,
\end{align*}
$$
where

- $\mathbf{D} = \text{diag}(d_i)\in\mathbb R^{n\times n}$ with $d_i = p/r_i$ if $r_i > 0$ and $d_i = 0$ if $r_i = 0$. $\mathbf{D}$ will be `rinv` in coding.
- $\mathbf{z} = (z_i)\in\mathbb R^n$ with $z_i = (1-p)/n$ if $r_i > 0$ and $z_i = 1/n$ if $r_i = 0$.

Note that $\mathbf{DA}$ is sparse and $\mathbf{z}\mathbf 1_n^T$ (outer product matrix) is of rank 1.

## Q2 Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.

## Q3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [4]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

[ Info: verify download of index files...
[ Info: reading database
[ Info: adding metadata...
[ Info: adding svd data...
[ Info: writing database
[ Info: used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [5]:
# connectivity matrix
A = md.A

916428×916428 SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎣⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎦

Compute summary statistics:  

* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  

* number of web pages

* number of edges (web links)

* number of dangling nodes (pages with no out links)

* histogram of in-degrees  

* list the top 20 pages with the largest in-degrees?  

* histogram of out-degrees

* which the top 20 pages with the largest out-degrees?

* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

**Solution**

Use `Base.summarysize` function to compute how much memory `A` takes in MB.

In [6]:
println(round(Base.summarysize(A) / (1024^2), digits = 2), " MB")

50.81 MB


If converted to `Matrix{Float64}`, then the memory usage of `A` will be $916428^2 * 8(\text{byte/double precision})$, i.e.,

In [7]:
println(round(916428^2 * 8 / (1024^4), digits = 2), " TB")

6.11 TB


The number of web pages corresponds to the order of `A`.

In [8]:
println(size(A, 1), " pages") # or size(A, 2)

916428 pages


The number of edges (web links) corresponds to the number of nonzero (or one) elements in `A`. There are at least three ways to get this:

In [9]:
@show count(!iszero, A)
@show nnz(A) # using SparseArrays
@show count(isone, A)
nothing

count(!iszero, A) = 5105039
nnz(A) = 5105039
count(isone, A) = 5105039


By the way, `nnz(A)` is much faster than `count(!iszero, A)` and `count(isone, A)`.

In [10]:
@show count(!iszero, A)
@benchmark count(!iszero, $A)

count(!iszero, A) = 5105039


BenchmarkTools.Trial: 7954 samples with 1 evaluation.
 Range (min … max):  615.667 μs …   9.282 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     616.583 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   626.212 μs ± 156.355 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▅▄▃▂▁▁                                                      ▁
  ████████████▇▆▅▆▅▅▆▅▅▁▅▄▄▅▄▄▃▃▃▃▁▄▃▄▃▃▄▁▁▄▁▁▃▁▁▁▃▁▁▁▁▄▃▁▁▁▃▅█ █
  616 μs        Histogram: log(frequency) by time        698 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [11]:
@show nnz(A)
@benchmark nnz(A)

nnz(A) = 5105039


BenchmarkTools.Trial: 10000 samples with 992 evaluations.
 Range (min … max):  32.930 ns …  3.706 μs  ┊ GC (min … max): 0.00% … 98.39%
 Time  (median):     38.138 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   37.736 ns ± 42.071 ns  ┊ GC (mean ± σ):  1.50% ±  1.38%

   ▆▃    ▆█▁                           ▁  ▄▅▂                  
  ▇██▄▄▃▄███▅▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▆▄▂▂▂▂▅█▅▇███▆▄▃▂▂▂▂▁▁▁▁▂▁▂▂▁ ▃
  32.9 ns         Histogram: frequency by time        43.3 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

The number of dangling nodes (pages with no out links) corresponds to the number of rows that contain only zero elements.

In [12]:
## count zeros in row sums (out-degree)
println(count(iszero, sum(A, dims = 2)), " pages have no out-links")

176974 pages have no out-links


Histogram of in-degrees. $\sum_i a_{ij}$ is the *in*-degree of page $j$.

In [13]:
in_degrees = sum(A, dims = 1)
histogram(in_degrees, title = "In-degree distribution")

                              In-degree distribution           
                    ┌                                        ┐ 
   [   0.0,  500.0) ┤███████████████████████████████  916 114  
   [ 500.0, 1000.0) ┤▏ 180                                     
   [1000.0, 1500.0) ┤▏ 32                                      
   [1500.0, 2000.0) ┤▏ 20                                      
   [2000.0, 2500.0) ┤▏ 16                                      
   [2500.0, 3000.0) ┤▏ 20                                      
   [3000.0, 3500.0) ┤▏ 18                                      
   [3500.0, 4000.0) ┤▏ 10                                      
   [4000.0, 4500.0) ┤▏ 9                                       
   [4500.0, 5000.0) ┤▏ 4                                       
   [5000.0, 5500.0) ┤▏ 4                                       
   [5500.0, 6000.0) ┤  0                                       
   [6000.0, 6500.0) ┤▏ 1                                       
                    └                   

We see that most pages can be visited from at most 500 links. Let's zoom in.

In [14]:
histogram(filter(x -> x ≤ 500, in_degrees[:]), title = "In-degree distribution (≤ 500)") 

                        In-degree distribution (≤ 500)       
                  ┌                                        ┐ 
   [  0.0,  50.0) ┤███████████████████████████████  904 305  
   [ 50.0, 100.0) ┤▍ 8 263                                   
   [100.0, 150.0) ┤▏ 1 952                                   
   [150.0, 200.0) ┤▏ 649                                     
   [200.0, 250.0) ┤▏ 375                                     
   [250.0, 300.0) ┤▏ 225                                     
   [300.0, 350.0) ┤▏ 116                                     
   [350.0, 400.0) ┤▏ 96                                      
   [400.0, 450.0) ┤▏ 69                                      
   [450.0, 500.0) ┤▏ 64                                      
                  └                                        ┘ 
                                   Frequency                 

Almost all pages have in-degrees of at most 50. Next, extract top 20 pages with the largest in-degrees:

In [15]:
## Top 20 largest in-degrees
top20_in = sortperm(vec(in_degrees), rev = true)[1:20]
hcat(top20_in, in_degrees[top20_in])

20×2 Matrix{Int64}:
 537040  6326
 597622  5354
 504141  5271
 751385  5182
  32164  5097
 885606  4847
 163076  4731
 819224  4620
 605857  4550
 828964  4484
 551830  4220
  41910  4219
 558792  4206
 459075  4187
 407611  4180
 213433  4084
 765335  4015
 384667  4010
 173977  3988
 687326  3956

The first column represents the page index and the second column represents in-degrees.

Histogram of out-degrees. $r_i = \sum_j a_{ij}$ is the *out*-degree of page $j$.

In [16]:
out_degrees = sum(A, dims = 2)
histogram(out_degrees, title = "Out-degree distribution")

                            Out-degree distribution          
                  ┌                                        ┐ 
   [  0.0,  20.0) ┤███████████████████████████████  891 798  
   [ 20.0,  40.0) ┤▊ 22 628                                  
   [ 40.0,  60.0) ┤▏ 1 329                                   
   [ 60.0,  80.0) ┤▏ 371                                     
   [ 80.0, 100.0) ┤▏ 124                                     
   [100.0, 120.0) ┤▏ 86                                      
   [120.0, 140.0) ┤▏ 29                                      
   [140.0, 160.0) ┤▏ 15                                      
   [160.0, 180.0) ┤▏ 15                                      
   [180.0, 200.0) ┤▏ 8                                       
   [200.0, 220.0) ┤▏ 7                                       
   [220.0, 240.0) ┤▏ 3                                       
   [240.0, 260.0) ┤▏ 8                                       
   [260.0, 280.0) ┤▏ 3                                       
   [280.

Most pages have no more than 20 different links. Let's zoom in.

In [17]:
histogram(filter(x -> x ≤ 20, out_degrees[:]), title = "Out-degree distribution (≤ 500)") 

                      Out-degree distribution (≤ 500)      
                ┌                                        ┐ 
   [ 0.0,  1.0) ┤███████████████████████████████  176 974  
   [ 1.0,  2.0) ┤██████████████████████▍ 127 937           
   [ 2.0,  3.0) ┤████████████████▋ 95 253                  
   [ 3.0,  4.0) ┤███████████▌ 65 976                       
   [ 4.0,  5.0) ┤█████████▍ 53 560                         
   [ 5.0,  6.0) ┤████████▎ 46 771                          
   [ 6.0,  7.0) ┤███████▎ 40 993                           
   [ 7.0,  8.0) ┤██████▋ 38 491                            
   [ 8.0,  9.0) ┤██████▌ 37 187                            
   [ 9.0, 10.0) ┤█████▋ 32 666                             
   [10.0, 11.0) ┤█████▎ 29 096                             
   [11.0, 12.0) ┤████▌ 25 656                              
   [12.0, 13.0) ┤███▉ 22 549                               
   [13.0, 14.0) ┤███▍ 19 156                               
   [14.0, 15.0) ┤██▉ 17 018             

The left-skewed distribution makes sense. Extract top 20 pages with the largest out-degrees;

In [18]:
## Top 20 largest out-degrees
top20_out = sortperm(vec(out_degrees), rev = true)[1:20]
hcat(top20_out, out_degrees[top20_out])

20×2 Matrix{Int64}:
 506743  456
 203749  372
 305230  372
 768092  330
 808644  277
 412411  268
 600480  265
 376429  258
 156951  257
 885729  256
 667585  253
 685696  248
 282141  247
 598189  245
 579315  244
 411594  231
 321092  229
 838279  225
 302734  216
 915274  213

Finally, we visualize the sparsity pattern of `A[1:10000, 1:10000]`. 

In [19]:
spy(A[1:10000, 1:10000])

          ┌──────────────────────────────────────────┐    
        1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
          │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
          │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
          │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
          │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
          │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
          │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
          │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
          │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
          │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
          │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
          │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
          │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠈⠀⠀⠀⠀│    
          │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
          │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠀⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
          │⠈⠀⠀⠄⠀⠀⡀⠈⠀⠀⠙⠀⠀⠀⢀⢀⠂⠀⠀⡄⢀⠀⠠⠀⠂⠀⠀⠐⠀⠀⠸⠀⠀⠀⠈⠠⠀⠠⠁⠀⠂⠀│  

## Q4 (5 pts) Dense linear algebra? 

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer. 

**Solution**

As mentioned in Q2, if we approach this problem as a linear system, our goal is to solve the linear equation
$$
\mathbf{P}^T\mathbf{x} = \mathbf{x}  \quad \text{s.t.} \quad\mathbf 1^T\mathbf x = 1.
$$
Replacing the first equation by $\mathbf 1^T\mathbf x = 1$ gives another linear equation without constraint
$$
\mathbf{P}_{\text{new}}^T\mathbf{x} = \mathbf{b},
$$
where $\mathbf{P}_{\text{new}}$ has $\mathbf 1_n$ in the first column and the same elements in the other $(n-1)$ columns as those in $\mathbf{P}$, and $\mathbf{b} = (1, x_2, \ldots, x_n)^T$.

Given LU decomposition $\mathbf{P}_{\text{new}}^T = \mathbf{LU}$, we first solve $\mathbf{L}\mathbf{y} = \mathbf{b}$ and then solve $\mathbf{U}\mathbf{x} = \mathbf{y}$ in order. Since $\mathbf{L}$ and $\mathbf{U}$ are of the same size as $\mathbf{P}_{\text{new}}$ and $\mathbf{L}$ and $\mathbf{U}$ require separately, we can store them in-place within the memory space of $\mathbf{P}_{\text{new}}$, which can be also stored in `full(A)`. Therefore, (1) the memory usage should be $916428^2\times 8 / 1024^4 = 6.11$ TB.

Regarding (2), we need approximately
$$
\frac{2}{3}n^3 = \frac{2}{3}(916428)^3 = 5.13 \times 10^{17} \text{ flops}.
$$
My computer (Apple M1 chip) has a theoretical CPU performance of 51.2 GFLOPS.

Therefore, assuming that LAPACK functions use only the CPU resource, it would take
$$
\frac{5.13 \times 10^{17}\text{(flops)}}{5.12\times 10^{10}\text{(flops/s)}} = 1.0 \times 10^7 \text{ seconds} \approx 116 \text{ days}
$$
to complete the task on my computer.

By the way, if we consider up to $\mathcal O(n^2)$, then the coefficient should be 1 not 2, as the LU desomposition takes
$$
2(n-1)^2 + 2(n-2)^2 + \cdots + 2\cdot 1^2 = 2\times\frac 1 6 (n-1)n(2n-1) \approx \frac{2}{3}n^3 - n^2 \text{ flops},
$$
providing
$$
\underbrace{\frac{2}{3}n^3 - n^2}_{\text{LU decomposition}} + \underbrace{2n^2}_{\text{solve } \mathbf{LUx} = \mathbf{b}} = \frac{2}{3}n^3 + n^2 \text{ flops}.
$$

## Q5 (75 pts) Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide are methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

**Solution**

For efficient computation, I expand $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ as follows:
$$
\begin{align*}
\mathbf{M} \mathbf{v}
&= (\mathbf{I} - \mathbf{A}^T\mathbf{D} - \mathbf{1}\mathbf{z}^T)\mathbf{v}
= \mathbf{v} - \mathbf{A}^T (\mathbf{D}\mathbf{v}) - \mathbf{1}(\mathbf{z}^T \mathbf{v})\\
\mathbf{M}^T \mathbf{v}
&= (\mathbf{I} - \mathbf{D}\mathbf{A} - \mathbf{z} \mathbf{1}^T)\mathbf{v}
= \mathbf{v} - \mathbf{D} (\mathbf{A}\mathbf{v}) - \mathbf{z} (\mathbf{1}_n^T\mathbf{v}).
\end{align*}
$$

In [20]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T # teleportation parameter p
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    store_n   :: Vector{T}
    rinv      :: Vector{T}
    z         :: Vector{T}
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n = size(A, 1)
    # TODO: initialize and pre-allocate arrays
    store_n   = Vector{T}(undef, n)
    r         = vec(sum(A, dims = 2))
    # rinv      = [x > 0 ? telep / x : 0.0 for x in r] # D
    # z         = [x > 0 ? (1 - telep) / n : 1 / n for x in r] # z
    rinv      = Vector{T}(undef, n) # D
    z         = Vector{T}(undef, n) # z
    @inbounds @simd for i in 1:n
        rinv[i] = r[i] == 0 ? 0 : telep / r[i]
        z[i] = r[i] == 0 ? 1 / n : (1 - telep) / n
    end
    PageRankImPt(A, telep, store_n, rinv, z)
end

# matrix element access M[i, j]
Base.getindex(M::PageRankImPt, i, j) = (i == j) - M.rinv[j] * M.A[j, i] - M.z[j]
LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)

# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}
        ) where T <: AbstractFloat
    # TODO: implement mul!(out, M, v)
    out .= v
    M.store_n .= M.rinv .* v                          # Dv
    mul!(out, Transpose(M.A), M.store_n, -1, 1) # v - A'Dv
    out .-= dot(M.z, v)                         # v - A'Dv - z'v
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}
        ) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    # TODO: implement mul!(out, transpose(M), v)
    out .= v
    mul!(M.store_n, M.A, v)          # Av
    out .-= M.rinv .* M.store_n # v - DAv
    out .-= M.z .* sum(v)       # v - DAv - z(1'v)
    return out
end

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M} \mathbf{x} = \mathbf{0}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv.gz`. **Do not put this file in your Git**. You will lose points if you do. You can add a line `pgrksol.csv.gz` to your `.gitignore` file.

In [21]:
using CodecZlib, DelimitedFiles

isfile("pgrksol.csv.gz") || download("https://github.com/ucla-biostat-257/2024spring/raw/master/hw/hw4/pgrksol.csv.gz")
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

**Solution**

In [22]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

# @assert transpose(M) * ones(n) ≈ zeros(n)
@assert norm(transpose(M) * ones(n)) < 1e-12

In [23]:
#@assert M * xsol ≈ zeros(n)
@assert norm(M * xsol) < 1e-12

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate no memory.

In [24]:
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 257 samples with 1 evaluation.
 Range (min … max):  17.963 ms … 59.891 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     18.136 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   19.190 ms ±  3.531 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▃▃      ▁                                                   
  ███▇█▄▇▅▇█▆▇▄▄▄▁▁▁▁▁▁▄▅▄▁▁▁▁▁▁▁▁▁▁▁▁▅▁▄▄▁▁▁▁▁▁▄▁▁▄▁▁▁▁▁▁▁▁▄ ▅
  18 ms        Histogram: log(frequency) by time      33.4 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [25]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 245 samples with 1 evaluation.
 Range (min … max):  19.449 ms … 29.560 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     19.557 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   20.179 ms ±  1.689 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▃             ▁                                             
  ███▄▄▅▅▆▇▇▄▄▁▄▆█▅▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▄▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▄▄ ▅
  19.4 ms      Histogram: log(frequency) by time      29.4 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [26]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

20.0

**Hint**: My median run times are about 10 ms and memory allocations are 0 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [27]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(value = ([3.378342822237397e-5, 2.0710155392781265e-6, 3.663065985290286e-6, 7.527510785714565e-7, 8.633285997289003e-7, 1.769418252763817e-6, 2.431230382949412e-7, 6.368417180841316e-7, 4.7449737038222236e-7, 2.689548611137733e-7  …  3.167946974023204e-6, 9.726880410372258e-7, 6.588614478640726e-7, 7.737011774829105e-7, 2.431230382949412e-7, 1.6219204214483914e-6, 3.912130060690359e-7, 2.431230382949412e-7, 7.296033831434169e-6, 6.330939996774086e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821117064217091e-13,).
), time = 5.373405583, bytes = 1511212064, gctime = 0.354590542, gcstats = Base.GC_Diff(1511212064, 206, 0, 12434, 4, 228, 354590542, 4, 1))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [28]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is

In [29]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 3-4 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [30]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

(value = (ComplexF64[-1.6594172665673617e-13 + 0.0im], Vector{ComplexF64}[[0.005635826953806507 + 0.0im, 0.00034549143807822166 + 0.0im, 0.00061108084941251 + 0.0im, 0.00012557561626038967 + 0.0im, 0.00014402240532806543 + 0.0im, 0.00029517830503989113 + 0.0im, 4.0558328286807854e-5 + 0.0im, 0.00010623935784854332 + 0.0im, 7.915671116200263e-5 + 0.0im, 4.4867650667280535e-5 + 0.0im  …  0.0005284839899796174 + 0.0im, 0.00016226599147987417 + 0.0im, 0.00010991273837613363 + 0.0im, 0.00012907055855754628 + 0.0im, 4.0558328286807854e-5 + 0.0im, 0.00027057238743206664 + 0.0im, 6.52630274833646e-5 + 0.0im, 4.0558328286807854e-5 + 0.0im, 0.001217140660133287 + 0.0im, 0.0001056141551070459 + 0.0im]], ConvergenceInfo: one converged value after 7 iterations and 99 applications of the linear map;
norms of residuals are given by (9.574464573688717e-14,).
), time = 10.233869333, bytes = 2296352192, gctime = 0.409751959, gcstats = Base.GC_Diff(2296352192, 311, 0, 16773, 12, 233, 409751959, 5, 3))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [31]:
@assert abs(Real(vals[1])) < 1e-8

In [32]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

Arnoldi should be reasonably fast. The points you'll get is

In [33]:
clamp(20 / time_arnoldi * 20, 0, 20)

20.0

**Hint**: My runtime is about 6-7 seconds.

## Q6 (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees? 

## Q7 Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.